In [ ]:
import pickle
import matplotlib.pyplot as plt
from search_boxes import SearchResult
from typing import List, Dict
import numpy as np
import MDAnalysis as mda

exp_dir = "tmp/exp_cos_cutoff_5"

with open(f"{exp_dir}/results.pkl", "rb") as f:
    results: List[SearchResult] = pickle.load(f)

In [ ]:
similarity_values = []
modified_atoms = []
for result in results:
    similarity_values.append(result.similarity)
    modified_atoms.append(result.modified_atoms)

In [ ]:
sorted_values = sorted(similarity_values, reverse=True)

plt.figure(figsize=(10, 5))
plt.bar(range(len(sorted_values)), sorted_values)
plt.xlabel("Ranked Molecules")
plt.ylabel("Similarity")
plt.title("Sorted Similarity Scores")
plt.tight_layout()
#plt.savefig("tmp/exp/sorted_similarity_barplot.png")
plt.show()

In [ ]:
print(sorted_values[0])

In [ ]:
pairs = list(zip(similarity_values, modified_atoms))

# 排序取 top 10
top_pairs = sorted(pairs, key=lambda x: x[0], reverse=True)[:10]

# 拆分为两个列表
top_similarities = [x[0] for x in top_pairs]
top_atoms = [x[1] for x in top_pairs]

In [ ]:
labels = ['-'.join(map(str, group)) for group in top_atoms]

# 画图
plt.figure(figsize=(10, 5))
bars = plt.bar(range(len(top_similarities)), top_similarities, tick_label=labels)
plt.xlabel("Atom Groups")
plt.ylabel("Similarity")
plt.title("Similarity of Atom Groups")
plt.xticks(rotation=45)  # x轴文字旋转，防止重叠

# 添加数值标签
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 0.01,  # 上方略微留白
             f"{top_similarities[i]:.2f}",  # 保留两位小数
             ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
labels = ['-'.join(map(str, group)) for group in top_atoms]

# 画图
plt.figure(figsize=(10, 5))
plt.bar(range(len(top_similarities)), top_similarities, tick_label=labels)
plt.xlabel("Atom Groups")
plt.ylabel("Similarity")
plt.title("Similarity of Atom Groups")
plt.xticks(rotation=45)  # x轴文字旋转，防止重叠
plt.tight_layout()
plt.show()

In [ ]:
import os
from utils import calculate_rmsd, load_structure_data, copy_atom_group, select_nbr_mols, generate_grids
from utils.analyze_structure import select_ccl4_molecules, analyze_ccl4_structure, rotate_ccl4_molecules
from ARPDF import compare_ARPDF
from search_boxes import save_ccl4_result
import json
from dataclasses import asdict

In [ ]:
# 设置参考结构目录
reference_dir = "data/CCl4"
output_dir = os.path.join(exp_dir, "post_analysis")

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# 加载参考结构
u1_ref, u2_ref, modified_atoms_ref, polar_axis_ref = load_structure_data(reference_dir)
ARPDF_ref = np.load(os.path.join(exp_dir, "ARPDF_ref.npy"))
with open(os.path.join(exp_dir, "metadata.json"), "r") as f:
    metadata = json.load(f)
xy_range = metadata["search_info"]["parameters"]["grids_range"]
N, M = metadata["search_info"]["parameters"]["grids_shape"]
X, Y = generate_grids(xy_range, N, M)

In [ ]:
def get_ccl4_universe(universe, ccl4_mols):
    """
    Create a new universe containing only the selected CCl4 molecules
    """
    selected_atoms = universe.atoms[ccl4_mols]
    new_universe = mda.Merge(selected_atoms)
    new_universe.dimensions = universe.dimensions
    return new_universe


In [ ]:
# Select the two CCl4 molecules by modified universe
ccl4_mols_ref = select_ccl4_molecules(u2_ref, 519)
# analyze the original structure of the selected CCl4 molecules
ccl4_universe_ref = get_ccl4_universe(u1_ref, ccl4_mols_ref)
ref_res = analyze_ccl4_structure(
    u1_ref, 519, selected_indices=ccl4_mols_ref
)
print(ref_res)

# rotate and save the reference structure
nbr_indices = select_nbr_mols(u2_ref, modified_atoms_ref, nbr_distance=5.0)
nbr_group = rotate_ccl4_molecules(u2_ref, ccl4_mols_ref, nbr_indices, polar_axis_ref)
nbr_group.write(os.path.join(output_dir, 'ref_nbr.gro'))

In [ ]:
# 计算每个搜索结果的RMSD
rmsd_values: List[float] = []
for i, result in enumerate(results):
    # 获取搜索结果结构
    u2_search = result.modified_universe

    # 获取搜索结果的CCl4分子
    ccl4_mols_search = select_ccl4_molecules(u2_search, result.molecule)
    ccl4_universe_search = get_ccl4_universe(u1_ref, ccl4_mols_search)

    # 使用select_ccl4_molecules生成selection
    selection = [0, 1, 5, 6]

    # 计算RMSD
    rmsd = calculate_rmsd(ccl4_universe_search, ccl4_universe_ref, selection=selection) # TODO: 需要修改
    rmsd_values.append(rmsd)
    print(f"Result {i+1} RMSD: {rmsd:.3f} Å, similarity: {result.similarity:.3f}")

In [ ]:
# 将RMSD值与相似度值进行比较
plt.figure(figsize=(12, 6))
plt.scatter(similarity_values, rmsd_values, alpha=0.6)
plt.xlabel('Similarity')
plt.ylabel('RMSD (Å)')
plt.title('Relationship between Similarity and RMSD')

# 添加趋势线
z = np.polyfit(similarity_values, rmsd_values, 1)
p = np.poly1d(z)
plt.plot(similarity_values, p(similarity_values), "r--", alpha=0.8)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'similarity_vs_rmsd.png'))
plt.show()

In [ ]:
# Filter structures with similarity > 0.8 and RMSD > 1.0
filtered_structures = []
for i, (result, rmsd) in enumerate(zip(results, rmsd_values)):
    similarity = result.similarity
    if similarity > 0.8 and rmsd > 1.0:
        # Get the search result structure
        u2_search = result.modified_universe
        # ARPDF_search[ARPDF_search > 0] = 0
        fig = compare_ARPDF(result.ARPDF, ARPDF_ref, grids_XY=(X, Y), cos_sim=similarity, show_range=8.0)
        fig.savefig(os.path.join(output_dir, f'filtered_structure_{i}.png'))
        ccl4_mols_search = select_ccl4_molecules(u2_search, result.modified_atoms[0])
        
        # Analyze the structure
        analysis = analyze_ccl4_structure(
            u1_ref,
            result.modified_atoms[0],
            selected_indices=ccl4_mols_search
        )
        
        # Save structure information
        filtered_structures.append({
            'index': i,
            'similarity': float(similarity),
            'rmsd': float(rmsd),
            'analysis': asdict(analysis),
            'structure_path': os.path.join(output_dir, f'filtered_structure_{i}.gro').replace("\\", "/")
        })
        
        # Save the structure file
        save_ccl4_result(result, os.path.join(output_dir, f'filtered_structure_{i}.gro'), nbr_distance=5.0)

# Save filtered results to JSON
if filtered_structures:
    with open(os.path.join(output_dir, 'filtered_structures.json'), 'w') as f:
        json.dump(filtered_structures, f, indent=4)
    
    print(f"\nFound {len(filtered_structures)} structures with similarity > 0.8 and RMSD > 1.0")
    for structure in filtered_structures:
        print(f"Structure {structure['index']}: similarity={structure['similarity']:.3f}, RMSD={structure['rmsd']:.3f}Å")
else:
    print("\nNo structures found with similarity > 0.8 and RMSD > 1.0")


In [ ]:
# Filter structures based on geometric criteria
filtered_results = []
for i, result in enumerate(results):
    # Get the search result structure
    u2_search = result.modified_universe
    ccl4_mols_search = select_ccl4_molecules(u2_search, result.modified_atoms[0])
    # Analyze the structure
    analysis = analyze_ccl4_structure(
        u1_ref, 
        result.modified_atoms[0], 
        selected_indices=ccl4_mols_search
    )
    
    # Calculate differences from reference structure
    theta_diff = abs(analysis.theta_CL_A_CL_B - ref_res.theta_CL_A_CL_B)
    dist_C_A_C_B_diff = abs(analysis.dist_C_A_C_B - ref_res.dist_C_A_C_B)
    dist_CL_A_CL_B_diff = abs(analysis.dist_CL_A_CL_B - ref_res.dist_CL_A_CL_B)
    
    # Apply filtering criteria
    if (theta_diff < 10 and 
        dist_C_A_C_B_diff < 0.3 and 
        dist_CL_A_CL_B_diff < 0.3):
        filtered_results.append({
            'index': i,
            'similarity': result.similarity,
            'theta_diff': theta_diff,
            'dist_C_A_C_B_diff': dist_C_A_C_B_diff,
            'dist_CL_A_CL_B_diff': dist_CL_A_CL_B_diff,
            'analysis': analysis
        })

# Print filtered results
print(f"\nFound {len(filtered_results)} structures matching criteria:")
for res in filtered_results:
    print(f"Result {res['index']+1}:")
    print(f"  Similarity: {res['similarity']:.3f}")
    print(f"  Theta difference: {res['theta_diff']:.1f}°")
    print(f"  C_A_C_B distance difference: {res['dist_C_A_C_B_diff']:.3f} Å")
    print(f"  CL_A_CL_B distance difference: {res['dist_CL_A_CL_B_diff']:.3f} Å")

# Analyze similarity distribution of filtered structures
if filtered_results:
    filtered_similarities = [res['similarity'] for res in filtered_results]
    print("\nSimilarity statistics for filtered structures:")
    print(f"  Mean: {np.mean(filtered_similarities):.3f}")
    print(f"  Std: {np.std(filtered_similarities):.3f}")
    print(f"  Min: {np.min(filtered_similarities):.3f}")
    print(f"  Max: {np.max(filtered_similarities):.3f}")
    
    # Plot similarity distribution
    plt.figure(figsize=(8, 5))
    plt.hist(filtered_similarities, bins=10, alpha=0.7)
    plt.xlabel('Similarity')
    plt.ylabel('Count')
    plt.title('Similarity Distribution of Filtered Structures')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'filtered_similarity_dist.png'))
    plt.show()
else:
    print("\nNo structures matched the filtering criteria")
